In [ ]:
# Imports
import pyETL.toolbox.nascar_toolbox as nascar_toolbox
import camelot
import pandas as pd

process_type = "Entry"

In [2]:
report_page_columns = {
   "Entry": [
      ("", "EntryNumber"),
      ("Car", "CarNumber"),
      ("Driver", "Driver Name"),
      ("Manufacturer", "Manufacturer"),
      ("Sponsor", "Sponsor"),
      ("Owner", "Owner Name"),
      ("Crew Chief", "Crew Chief"),
   ],
   "EntryAlt": [
      ("", "EntryNumber"),
      ("Entry Veh#", "CarNumber"),
      ("Driver", "Driver Name"),
      ("Organization", "Owner Name"),
      ("Crew Chief", "Crew Chief"),
      ("Veh Mfg", "Manufacturer"),
      ("Sponsor", "Sponsor"),
   ]
}

In [3]:
# Get dataframes from all pages of PDF
stage_race_metrics_df = nascar_toolbox.create_empty_race_metrics_df()
stage_entry_metrics_df = nascar_toolbox.create_empty_entry_metrics_df()
audit_log_df = nascar_toolbox.create_empty_audit_log_df()

# last_run = nascar_toolbox.get_last_run(report_type)
report_dir = nascar_toolbox.get_data_dir(process_type)

for report in report_dir.glob("**/*.pdf"):
   report_file = report.relative_to(report_dir)
   report_dfs = camelot.read_pdf(report, flavor="stream", pages="all")
   for i, d in enumerate(report_dfs):
      report_df = d.df

      first_entry_i = report_df[report_df[0] == "1"].index[0]
      page_definition = nascar_toolbox.check_for_report_page_definiton(process=process_type, report_columns=report_page_columns, report_file=report_file, table_i=i, report_df=report_df, first_entry_i=first_entry_i, audit_log_df=audit_log_df)
      if page_definition["result"]:
         page_columns = [c[1] for c in report_page_columns[page_definition["found_page"]]]
         report_df.columns = page_columns

         # Remove rows before first actual entry
         first_entry_i = report_df[report_df["EntryNumber"] == "1"].index[0]
         report_df = report_df[first_entry_i:].reset_index(drop=True)

         # Fix cases where values were wrapped to the next line (i.e. one entry is multiple rows in df)
         # Join column values of row without entry number to previous row
         non_entry_indices = report_df.loc[report_df["EntryNumber"] == ""].index
         for nei in reversed(non_entry_indices):
            temp = report_df[nei - 1: nei + 1].copy()
            temp["gb"] = nei - 1
            report_df.update(temp.groupby("gb", sort=False)[page_columns].agg(" ".join).map(lambda x: str(x).strip()))#.apply(lambda x: x.str.strip))
            report_df = report_df.drop([nei])

         # TODO
         # Remove non numerical entry numbers
         # Catch column values getting join (i.e. blank columns)
         # Car number with * - non-chartered cars
         # Drivers with # - rookies
         # Drivers with (i) - point ineligible

         report_df = report_df.reset_index(drop=True)
         # print(report_df.to_string())

